In [71]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import utility_functions as uf
import utility_functions2 as uf
import string
import itertools
import timeit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
n = 10000

In [73]:
name_gen = map(''.join, itertools.islice(itertools.product('ABCDE',repeat=10), n))
names = list(name_gen)

In [74]:
data = np.triu(np.random.random((n, n)))
data = (data + data.T) / 2

In [75]:
df = pd.DataFrame(data, index=names, columns=names)
df.astype(np.float16)
df

,AAAAAAAAAA,AAAAAAAAAB,AAAAAAAAAC,AAAAAAAAAD,AAAAAAAAAE,AAAAAAAABA,AAAAAAAABB,AAAAAAAABC,AAAAAAAABD,AAAAAAAABE,...,AAAADAEEDA,AAAADAEEDB,AAAADAEEDC,AAAADAEEDD,AAAADAEEDE,AAAADAEEEA,AAAADAEEEB,AAAADAEEEC,AAAADAEEED,AAAADAEEEE
AAAAAAAAAA,0.228630,0.481428,0.257636,0.433829,0.393190,0.465618,0.407889,0.251970,0.027475,0.385064,...,0.409511,0.319103,0.442446,0.130805,0.317029,0.021887,0.180014,0.035975,0.365029,0.169212
AAAAAAAAAB,0.481428,0.736787,0.153195,0.389719,0.399968,0.310100,0.310933,0.064749,0.063583,0.292969,...,0.338255,0.117887,0.294068,0.132577,0.097951,0.038923,0.317512,0.405169,0.182635,0.393358
AAAAAAAAAC,0.257636,0.153195,0.674979,0.295434,0.088387,0.206130,0.213188,0.485287,0.207868,0.150774,...,0.073197,0.332283,0.399719,0.485960,0.256963,0.312217,0.366727,0.217089,0.019931,0.469889
AAAAAAAAAD,0.433829,0.389719,0.295434,0.059139,0.174820,0.487322,0.024741,0.257355,0.442930,0.420913,...,0.217570,0.284448,0.417498,0.234349,0.251300,0.073010,0.260162,0.202520,0.493647,0.127947
AAAAAAAAAE,0.393190,0.399968,0.088387,0.174820,0.652269,0.337629,0.306583,0.291289,0.459278,0.299842,...,0.354093,0.021595,0.069625,0.447977,0.228455,0.154456,0.486010,0.332740,0.365369,0.466113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AAAADAEEEA,0.021887,0.038923,0.312217,0.073010,0.154456,0.464723,0.241830,0.161788,0.085303,0.264557,...,0.243945,0.241495,0.213635,0.297826,0.137287,0.275536,0.304617,0.108301,0.461219,0.284371
AAAADAEEEB,0.180014,0.317512,0.366727,0.260162,0.486010,0.026279,0.459073,0.332305,0.098256,0.137417,...,0.101471,0.305833,0.351376,0.475130,0.334286,0.304617,0.511551,0.345684,0.112143,0.479133
AAAADAEEEC,0.035975,0.405169,0.217089,0.202520,0.332740,0.323396,0.298856,0.164755,0.078325,0.493734,...,0.328273,0.209307,0.167358,0.162144,0.000517,0.108301,0.345684,0.921598,0.001354,0.443158
AAAADAEEED,0.365029,0.182635,0.019931,0.493647,0.365369,0.344245,0.188248,0.176454,0.242241,0.118606,...,0.213265,0.241180,0.189479,0.097953,0.014935,0.461219,0.112143,0.001354,0.659214,0.382929


In [66]:
# %time df.to_csv('Output/similarity_to_csv.csv.gz', compression='gzip')

CPU times: user 4.41 s, sys: 62.5 ms, total: 4.47 s
Wall time: 4.48 s


In [67]:
# %time np.savetxt('Output/similarity_savetext.txt', data.astype(np.float32), delimiter='\t')

CPU times: user 641 ms, sys: 0 ns, total: 641 ms
Wall time: 655 ms


In [68]:
# %time np.savez_compressed('Output/similarity_savez_compressed.npz', data.astype(np.float32), delimiter='\t')

CPU times: user 156 ms, sys: 15.6 ms, total: 172 ms
Wall time: 177 ms


In [76]:
%time np.savez_compressed('Output/similarity_triu_savez_compressed.npz', data=np.triu(data.astype(np.float32)))

CPU times: user 8.48 s, sys: 1.09 s, total: 9.58 s
Wall time: 9.63 s


In [78]:
%%time
with np.load('Output/similarity_triu_savez_compressed.npz') as data_load:
    data = data_load['data']
    data = data + data.T - np.diag(data.diagonal())
    print(data)

[[0.22863002 0.48142788 0.25763556 ... 0.03597526 0.36502942 0.1692117 ]
 [0.48142788 0.73678714 0.15319528 ... 0.405169   0.18263459 0.39335838]
 [0.25763556 0.15319528 0.6749793  ... 0.21708888 0.01993083 0.46988875]
 ...
 [0.03597526 0.405169   0.21708888 ... 0.92159796 0.00135357 0.4431581 ]
 [0.36502942 0.18263459 0.01993083 ... 0.00135357 0.65921444 0.38292888]
 [0.1692117  0.39335838 0.46988875 ... 0.4431581  0.38292888 0.18439339]]
CPU times: user 2.67 s, sys: 1.22 s, total: 3.89 s
Wall time: 3.9 s
